<a href="https://colab.research.google.com/github/aSafarpoor/SCLFB/blob/main/SCLFB_phase1_one_layer_CL_phase_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
import pickle
import random
import json
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import math
import cv2
from google.colab.patches import cv2_imshow
import os

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
tqdm.pandas()

#go to path

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd drive/MyDrive/MSc/SCLAD/
# %mkdir FB15k237
%cd FB15k237

/content/drive/MyDrive/MSc/SCLAD
/content/drive/MyDrive/MSc/SCLAD/FB15k237


#read data

In [6]:
# !wget https://raw.githubusercontent.com/intfloat/SimKGC/main/data/FB15k237/FB15k_mid2name.txt
# !wget https://raw.githubusercontent.com/intfloat/SimKGC/main/data/FB15k237/FB15k_mid2description.txt
# !wget https://raw.githubusercontent.com/intfloat/SimKGC/main/data/FB15k237/test.txt
# !wget https://raw.githubusercontent.com/intfloat/SimKGC/main/data/FB15k237/train.txt
# !wget https://raw.githubusercontent.com/intfloat/SimKGC/main/data/FB15k237/valid.txt

In [7]:
!ls

 des.p			     valid.txt
 FB15k_mid2description.txt  'x part1 p&n samples.pkl'
 FB15k_mid2name.txt	    'x part2 p&n samples.pkl'
 sim.pkl		    'x test part1 p&n samples.pkl'
 test.pkl		    'y part1 p&n samples.pkl'
 test.txt		    'y part2 p&n samples.pkl'
 train.txt		    'y test part1 p&n samples.pkl'


In [8]:
with open('des.p','rb') as f:
     des = pickle.load(f)

des.head(3)

,namecode,description,title,text,embedded
0,/m/06rf7,schleswigholstein northernmost sixteen states ...,Schleswig-Holstein,schleswigholstein schleswigholstein northernmo...,"[-0.5489803, 0.16440539, 0.66058505, -0.037717..."
1,/m/0c94fn,gary roger rydstrom american sound designer di...,Gary_Rydstrom,gary_rydstrom gary roger rydstrom american sou...,"[-0.36805353, 1.2219998, 0.22228242, 0.5203364..."
2,/m/016ywr,jeremy john irons english actor receiving clas...,Jeremy_Irons,jeremy_irons jeremy john irons english actor r...,"[-0.8121141, 0.7617216, 0.43965185, -0.0815964..."


In [9]:
dfname = []
for line in open('FB15k_mid2name.txt', 'r'):
    dfname.append(line.replace('\n','').split('\t'))
dfname = pd.DataFrame(dfname)
dfname = dfname.rename(columns={0: "namecode", 1: "title"})
dfname.head(5)

,namecode,title
0,/m/06rf7,Schleswig-Holstein
1,/m/0c94fn,Gary_Rydstrom
2,/m/016ywr,Jeremy_Irons
3,/m/01yjl,Chicago_Cubs
4,/m/02hrh1q,Actor-GB


In [10]:
train = []
for line in open('train.txt', 'r'):
    train.append(line.replace('\n','').split('\t'))
train = pd.DataFrame(train)
train = train.rename(columns={0: "namecode1", 1: "title", 2: "namecode2"})
train.head(5)

,namecode1,title,namecode2
0,/m/027rn,/location/country/form_of_government,/m/06cx9
1,/m/017dcd,/tv/tv_program/regular_cast./tv/regular_tv_app...,/m/06v8s0
2,/m/07s9rl0,/media_common/netflix_genre/titles,/m/0170z3
3,/m/01sl1q,/award/award_winner/awards_won./award/award_ho...,/m/044mz_
4,/m/0cnk2q,/soccer/football_team/current_roster./sports/s...,/m/02nzb8


In [11]:
test = []
for line in open('test.txt', 'r'):
    test.append(line.replace('\n','').split('\t'))
test = pd.DataFrame(test)
test = test.rename(columns={0: "namecode1", 1: "title", 2: "namecode2"})
test.head(5)

,namecode1,title,namecode2
0,/m/08966,/travel/travel_destination/climate./travel/tra...,/m/05lf_
1,/m/01hww_,/music/performance_role/regular_performances./...,/m/01q99h
2,/m/09v3jyg,/film/film/release_date_s./film/film_regional_...,/m/0f8l9c
3,/m/02jx1,/location/location/contains,/m/013t85
4,/m/02jx1,/location/location/contains,/m/0m0bj


In [12]:
valid = []
for line in open('valid.txt', 'r'):
    valid.append(line.replace('\n','').split('\t'))
valid = pd.DataFrame(valid)
valid = valid.rename(columns={0: "namecode1", 1: "title", 2: "namecode2"})
valid.head(5)

,namecode1,title,namecode2
0,/m/07pd_j,/film/film/genre,/m/02l7c8
1,/m/06wxw,/location/location/time_zones,/m/02fqwt
2,/m/01t94_1,/people/person/spouse_s./people/marriage/type_...,/m/04ztj
3,/m/02xcb6n,/award/award_category/winners./award/award_hon...,/m/04x4s2
4,/m/07f_7h,/film/film/release_date_s./film/film_regional_...,/m/04gzd


In [13]:
def reset_index(df):
    df = df.reset_index()
    del df['index']
    return df

test = reset_index(test)
train = reset_index(train)
valid = reset_index(valid)

#preprocessing

In [14]:
nodes = list(set(des.namecode))

In [15]:
len(nodes)

14904

In [16]:
des.head(5)

,namecode,description,title,text,embedded
0,/m/06rf7,schleswigholstein northernmost sixteen states ...,Schleswig-Holstein,schleswigholstein schleswigholstein northernmo...,"[-0.5489803, 0.16440539, 0.66058505, -0.037717..."
1,/m/0c94fn,gary roger rydstrom american sound designer di...,Gary_Rydstrom,gary_rydstrom gary roger rydstrom american sou...,"[-0.36805353, 1.2219998, 0.22228242, 0.5203364..."
2,/m/016ywr,jeremy john irons english actor receiving clas...,Jeremy_Irons,jeremy_irons jeremy john irons english actor r...,"[-0.8121141, 0.7617216, 0.43965185, -0.0815964..."
3,/m/01yjl,chicago cubs professional baseball team locate...,Chicago_Cubs,chicago_cubs chicago cubs professional basebal...,"[-0.5676567, 0.1930804, -0.13923785, 0.4089947..."
4,/m/02hrh1q,actor person portraying character dramatic com...,Actor-GB,actorgb actor person portraying character dram...,"[0.30152893, 0.9061507, 0.70486355, 0.19482128..."


In [17]:
tempa = des.namecode
tempb = des.embedded
d = {}
for i in range(len(tempa)):
    d[tempa[i]]=[np.array(tempb[i])]    

##train:

In [18]:
list(d.keys())[0],np.array(d[list(d.keys())[0]]).shape

('/m/06rf7', (1, 768))

## function

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
def similarity(x,y):
    return cosine_similarity([x],[y])

In [21]:
import warnings
warnings.filterwarnings("ignore")

# create dataset for nn

In [22]:
# xtrain = []
# ytrain = []

# xtest = []
# ytest = []

# for i in tqdm(range(int(len(train)/2))):
#     row = train.loc[i]
#     try:
#         x1 = des[des.namecode==row[0]].namecode
#         x2 = des[des.namecode==row[2]].namecode
#         xtrain.append([row[0],row[2]])
#         ytrain.append(1)
#         while(True):
#             z = random.randint(0,len(des)-1)
#             if not ((train['namecode1'] == x1.values[0]) & (train['namecode2'] == des.loc[z][0])).any():
#                 z = des[des.namecode==des.loc[z][0]].description
#                 break
#         xtrain.append([row[0],z])
#         ytrain.append(0)
#     except:
#         pass

In [23]:
# with open('x part1 p&n samples.pkl', 'wb') as handle:
#     pickle.dump(xtrain, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('y part1 p&n samples.pkl', 'wb') as handle:
#     pickle.dump(ytrain, handle, protocol=pickle.HIGHEST_PROTOCOL)

# # with open('filename.pkl', 'rb') as handle:
# #     b = pickle.load(handle)

# next part

In [94]:
with open('x part1 p&n samples.pkl', 'rb') as handle:
    x1 = pickle.load(handle)

with open('x part2 p&n samples.pkl', 'rb') as handle:
    x2 = pickle.load(handle)

with open('y part1 p&n samples.pkl', 'rb') as handle:
    y1 = pickle.load(handle)

with open('y part2 p&n samples.pkl', 'rb') as handle:
    y2 = pickle.load(handle)

In [95]:
xtrain = x1[:] + x2[:]
ytrain = y1[:] + y2[:]

In [26]:
# xtest = []
# ytest = []

# xtest = []
# ytest = []

# for i in tqdm(range(int(len(test)/2))):
#     row = test.loc[i]
#     try:
#         x1 = des[des.namecode==row[0]].namecode
#         x2 = des[des.namecode==row[2]].namecode
#         xtest.append([row[0],row[2]])
#         ytest.append(1)
#         while(True):
#             z = random.randint(0,len(des)-1)
#             if not ((test['namecode1'] == x1.values[0]) & (test['namecode2'] == des.loc[z][0])).any():
#                 z = des[des.namecode==des.loc[z][0]].description
#                 break
#         xtest.append([row[0],z])
#         ytest.append(0)
#     except:
#         pass

In [27]:
# with open('x test part1 p&n samples.pkl', 'wb') as handle:
#     pickle.dump(xtest, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('y test part1 p&n samples.pkl', 'wb') as handle:
#     pickle.dump(ytest, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('x test part1 p&n samples.pkl', 'rb') as handle:
    xtest = pickle.load(handle)

with open('y test part1 p&n samples.pkl', 'rb') as handle:
    ytest = pickle.load(handle)

# add descriptions

In [28]:
des[:3]

,namecode,description,title,text,embedded
0,/m/06rf7,schleswigholstein northernmost sixteen states ...,Schleswig-Holstein,schleswigholstein schleswigholstein northernmo...,"[-0.5489803, 0.16440539, 0.66058505, -0.037717..."
1,/m/0c94fn,gary roger rydstrom american sound designer di...,Gary_Rydstrom,gary_rydstrom gary roger rydstrom american sou...,"[-0.36805353, 1.2219998, 0.22228242, 0.5203364..."
2,/m/016ywr,jeremy john irons english actor receiving clas...,Jeremy_Irons,jeremy_irons jeremy john irons english actor r...,"[-0.8121141, 0.7617216, 0.43965185, -0.0815964..."


In [47]:
i=9799
h = xtrain[i][0]
t = xtrain[i][1]
print(h,t)
# des[des.description==t.values[0]].embedded.values[0]

/m/04jpl 14155    deathcore genre extreme metal combines sounds ...
Name: description, dtype: object


In [92]:
xtrain = x1[:] + x2[:]
ytrain = y1[:] + y2[:]

In [115]:
badlist = []
for i in tqdm(range(len(xtest))):
    h = xtest[i][0]
    t = xtest[i][1]
    try:
        if len(h)!=768:
            h = des[des.namecode==h].embedded.values[0]
        if len(t)!=768:
            if type(t)!=str:
                t = t.values[0]
            if '/m' == t[:2]:
                t = des[des.namecode==t].embedded.values[0]
            else:
                t = des[des.description==t].embedded.values[0]

        xtest[i][0] = h
        xtest[i][1] = t
    except Exception as e:
        print(i,'----------> ',e)
        badlist.append(i)

  8%|▊         | 1617/20455 [00:06<02:08, 147.00it/s]

1592 ---------->  index 0 is out of bounds for axis 0 with size 0


  9%|▉         | 1797/20455 [00:07<01:40, 185.83it/s]

1753 ---------->  index 0 is out of bounds for axis 0 with size 0


 12%|█▏        | 2460/20455 [00:11<01:50, 162.77it/s]

...

 79%|███████▉  | 16248/20455 [01:18<00:17, 246.08it/s]

16211 ---------->  index 0 is out of bounds for axis 0 with size 0


 93%|█████████▎| 19010/20455 [01:31<00:09, 155.03it/s]

18992 ---------->  index 0 is out of bounds for axis 0 with size 0


100%|██████████| 20455/20455 [01:38<00:00, 208.49it/s]


In [122]:
Xtest = []
Ytest = []
for i in range(len(xtest)):
    if len(xtest[i])==2 and len(xtest[i][0])==768 and len(xtest[i][1])==768:
        Xtest.append(xtest[i])
        Ytest.append(ytest[i])
    else:
        pass

In [123]:
xtest = np.array(Xtest)
xtest.shape

(20441, 2, 768)

In [124]:
with open('xtest pn.pkl', 'wb') as handle:
    pickle.dump(xtest, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [125]:
with open('ytest pn.pkl', 'wb') as handle:
    pickle.dump(Ytest, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [126]:
len(xtest),len(Ytest)

(20441, 20441)